First, we manage all imports for the code.

In [2]:
!pip install gensim
!pip install pandas
!pip install nltk
!pip install install -Uq bertopic
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
#imports
import pandas as pd
import numpy as np
import random
import ast
from torch import nn
import torch
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from torch.utils.data import random_split, Subset, DataLoader
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import zipfile
import os
import shutil

import bertopic
from sentence_transformers import SentenceTransformer

/home/alfrodo/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Next we process the zipped data to csv. It is stored in .zip, because it is smaller on git and then locally it is extracted to csv.

In [4]:
with zipfile.ZipFile("data/dataset.zip", "r") as zip_ref:
    zip_ref.extractall("data")

for file_name in os.listdir("data/dataset"):
    source = "data/dataset/" + file_name
    destination = "data/" + file_name
    if os.path.isfile(source): shutil.move(source, destination)

os.rmdir("data/dataset")
del source, destination

Preprocessing of the datasets, the goal is to receive a table with the following columns: Name, Lyrics, Genre<br>
df Dataset - Name, Lyrics, Genre<br>
df3 Dataset - 10000 entrys of random selectet lyrics to train doc2vec
df4 Dataset - 28k~ entrys of songs

In [5]:

df = pd.read_csv('data/Spotify-2000.csv')
df = df[['Title', 'Top Genre']] #take only the name and genre
df2 = pd.read_csv('data/spotify_millsongdata.csv')
df4 = pd.read_csv('data/tcc_ceds_music.csv')
df4 = df4[['track_name','genre','lyrics']]

df['lyrics'] = '' # add column lyrics
#Now combine those two documents by the title
found = 0

for x, title in enumerate(df['Title']):
    #print(title)
    for y, title2 in enumerate(df2['song']):
        if title2.lower() == title.lower():
            df['lyrics'].iloc[x] = df2['text'].iloc[y]

print("finished combining..")

finished combining..


Collect 10000 random entrys of lyrics from the millsongdata Dataset

In [6]:
df3 = df2['text'].sample(n=10000)
#print(df3)
print('done')

done


In [7]:
#remove songs which were not in both datsets
songs_to_remove = []
for x, lyrics in enumerate(df['lyrics']):
    if lyrics == '':
        songs_to_remove.append(x)
df.drop(songs_to_remove, axis = 0, inplace = True)

print(df)

                  Title          Top Genre  \
0               Sunrise    adult standards   
1           Black Night         album rock   
3         The Pretender  alternative metal   
6     She Will Be Loved                pop   
8        Mr. Brightside        modern rock   
...                 ...                ...   
1988         Summertime    adult standards   
1989   Heartbreak Hotel    adult standards   
1990          Hound Dog    adult standards   
1991    Johnny B. Goode         blues rock   
1993     Blueberry Hill    adult standards   

                                                 lyrics  
0     You take away the breath I was keeping for sun...  
1     Black night is not right,  \r\nI don't feel so...  
3     I'm going to rent myself a house  \r\nIn the s...  
6     Beauty queen of only eighteen she  \r\nHad som...  
8     Coming out of my cage  \r\nAnd I've been doing...  
...                                                 ...  
1988  Summertime....and the livin' is eas

Further preprocessing of the lyrics itself. We remove the stopwords and punctuations with regex and stopwords from ntlk form df and df3

In [8]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stemmer = SnowballStemmer("english")
sw = stopwords.words('english')
punc_regex = r'[^\s\w]' #searches for everything thats not a word or space
stopword_regex = r'\b{0}\b'
space_regex = r'\s\s+'
newl_regex = r'\n|\r'

#print(df3.iloc[1])

#preprocessing of df
for x, lyrics in enumerate(df['lyrics']):
    txt = lyrics
    txt = re.sub(punc_regex, '', txt) #remove punuctuations
    for sword in sw:
        txt = re.sub(stopword_regex.format(sword), '', txt, flags=re.IGNORECASE) #remove every stopword
    txt = re.sub(newl_regex, '', txt)
    stemmed_txt = ''
    for word in txt.split():
        stemmed_txt += str(stemmer.stem(word.lower())) + " "
    
    df['lyrics'].iloc[x] = stemmed_txt

print(df)
print('finished preprocessing of df')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alfrodo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                  Title          Top Genre  \
0               Sunrise    adult standards   
1           Black Night         album rock   
3         The Pretender  alternative metal   
6     She Will Be Loved                pop   
8        Mr. Brightside        modern rock   
...                 ...                ...   
1988         Summertime    adult standards   
1989   Heartbreak Hotel    adult standards   
1990          Hound Dog    adult standards   
1991    Johnny B. Goode         blues rock   
1993     Blueberry Hill    adult standards   

                                                 lyrics  
0     take away breath keep sunris appear morn look ...  
1     black night right dont feel bright dont care s...  
3     im go rent hous shade freeway gonna pack lunch...  
6     beauti queen eighteen troubl alway help alway ...  
8     come cage ive fine gotta gotta want start kiss...  
...                                                 ...  
1988  summertimeand livin easi fish jumpi

In [9]:
#preprocessing of df3
for x, lyrics in enumerate(df3):
    txt = lyrics
    txt = re.sub(punc_regex, '', txt) #remove punuctuations
    for sword in sw:
        txt = re.sub(stopword_regex.format(sword), '', txt, flags=re.IGNORECASE) #remove every stopword
    txt = re.sub(newl_regex, '', txt)
    stemmed_txt = ''
    for word in txt.split():
        stemmed_txt += str(stemmer.stem(word.lower())) + " "
        
    df3.iloc[x] = stemmed_txt
    
print(df3)
print('finished preprocessing of df3')

29764    turn tide calm angri sea alon decid write symp...
34650    born nixon rais hell welfar child teamster dwe...
38399    girl sometim seem dont say thing act like im g...
4860     come your understand tell lie pretend believ s...
30581    round drive crazi watch run make lazi your tri...
                               ...                        
56020    im human sometim make mistak forgiv im gonna t...
56557    climb aboard butterfli take breez let worri fl...
56185    woman enough take man like babi know got make ...
7408     everybodi say let asham take round make differ...
44614    learn love assembl way today tomorrow alway we...
Name: text, Length: 10000, dtype: object
finished preprocessing of df3


In [10]:
#preprocessing of df4
for x, lyrics in enumerate(df4['lyrics']):
    txt = lyrics
    txt = re.sub(punc_regex, '', txt) #remove punuctuations
    for sword in sw:
        txt = re.sub(stopword_regex.format(sword), '', txt, flags=re.IGNORECASE) #remove every stopword
    
    txt = re.sub(newl_regex, '', txt)
    stemmed_txt = ''
    for word in txt.split():
        stemmed_txt += str(stemmer.stem(word.lower())) + " "
    df4['lyrics'].iloc[x] = stemmed_txt

print(df4)
print('finished preprocessing of df4')

                          track_name    genre  \
0               mohabbat bhi jhoothi      pop   
1                          i believe      pop   
2                                cry      pop   
3                           patricia      pop   
4                 apopse eida oneiro      pop   
...                              ...      ...   
28367                10 million ways  hip hop   
28368  ante up (robbin hoodz theory)  hip hop   
28369                  whutcha want?  hip hop   
28370                         switch  hip hop   
28371                         r.i.p.  hip hop   

                                                  lyrics  
0      hold time feel break feel untru convinc speak ...  
1      believ drop rain fall grow believ darkest nigh...  
2      sweetheart send letter goodby secret feel bett...  
3      kiss lip want stroll charm mambo chacha mering...  
4      till darl till matter know till dream live apa...  
...                                                  ... 

In [11]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

#display(df)

topic_model = bertopic.BERTopic(language='english', embedding_model=sentence_model,verbose=True)

topics, probs = topic_model.fit_transform(df["lyrics"])

print(topics)

Downloading: 100%|█████████████████████████████| 116/116 [00:00<00:00, 34.9kB/s]
Downloading: 100%|██████████████████████████| 39.3k/39.3k [00:00<00:00, 136kB/s]
Downloading: 100%|██████████████████████████| 90.9M/90.9M [02:07<00:00, 712kB/s]
Downloading: 100%|███████████████████████████| 53.0/53.0 [00:00<00:00, 14.9kB/s]
Downloading: 100%|█████████████████████████████| 112/112 [00:00<00:00, 42.0kB/s]
Downloading: 100%|████████████████████████████| 466k/466k [00:00<00:00, 688kB/s]
Downloading: 100%|██████████████████████████████| 350/350 [00:00<00:00, 142kB/s]
Downloading: 100%|██████████████████████████| 13.2k/13.2k [00:00<00:00, 109kB/s]
Downloading: 100%|████████████████████████████| 232k/232k [00:00<00:00, 428kB/s]
Downloading: 100%|██████████████████████████████| 349/349 [00:00<00:00, 122kB/s]


KeyError: 103

In [ ]:
print(topic_model.get_topic_info())

print(topic_model.get_topics()[1])

Tokenize the lyrics and create tagged Documents

In [13]:
tagged_data = []
nltk.download('punkt')

for i,d in enumerate(df4['lyrics']):
    tokenized_words = nltk.tokenize.word_tokenize(d)
    tagged_data.append(TaggedDocument(words=tokenized_words, tags=str(i)))

print(len(tagged_data))

[nltk_data] Downloading package punkt to /home/alfrodo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


28372


Setting up the Doc2Vec model

In [14]:
doc2vec_model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=1, epochs=30)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

print("Doc2Vec model finished training")

Doc2Vec model finished training


Building the DataLoader for the machine learning model

In [15]:
columns = ['Token'] + ['Target']
df_for_dataloader = pd.DataFrame(columns = columns)
df_for_dataloader.set_index(columns)

lookup_dict = {}
token_list = []
target_list = []
i = 0
for d in df4["genre"]:
    if d not in lookup_dict:
        lookup_dict[d] = i
        i+=1

for index, df_row in df4.iterrows():
    lyrics_tokenized = nltk.tokenize.word_tokenize(df_row["lyrics"])
    token = [doc2vec_model.infer_vector(lyrics_tokenized)]
    #print(token)
    one_hot_encoded_vector = []
    for x in lookup_dict.keys():
        if df_row["genre"] == x:
            one_hot_encoded_vector.append(1)
        else:
            one_hot_encoded_vector.append(0)
    target = [np.array(one_hot_encoded_vector)] # should be genre
    #row = pd.DataFrame([token + target], columns=['Token', 'Target'])
    #df_for_dataloader = pd.concat([df_for_dataloader, row])
    token_list.append(token)
    target_list.append(target)


Splitting the dataset into train and test (80,20)

In [16]:
data_train, data_test = torch.utils.data.random_split(list(zip(token_list, target_list)), [0.8, 0.2])
print(len(data_train))
print(len(data_test))
    
dataloader_train = torch.utils.data.DataLoader(data_train, batch_size=32)
dataloader_test = torch.utils.data.DataLoader(data_test, batch_size=32)


22698
5674


In [18]:
genre_test_count_dict = {}
for i in data_test:
  if np.argmax(i[1]) in genre_test_count_dict:
    genre_test_count_dict[np.argmax(i[1])] += 1
  else:
    genre_test_count_dict[np.argmax(i[1])] = 1

for i in genre_test_count_dict:
  print(i, genre_test_count_dict[i])

2 968
1 1090
5 811
0 1352
6 184
3 766
4 503


In [26]:
device = torch.device('cpu') #torch.device('cuda' if torch.cuda.is_available() else 'cpu') For faster training in the end

class SongModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(SongModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = input_size
        self.output_size = output_size
        self.layer_1 =   nn.Linear(self.input_size, self.hidden_size)
        self.layer_2 =   nn.Linear(self.hidden_size, self.hidden_size)
        self.layer_out = nn.Linear(self.hidden_size, self.output_size)
        self.softmax =   nn.Softmax()
        self.sigmoid =   nn.Sigmoid()
        self.tanh =      nn.Tanh()

        nn.init.xavier_uniform_(self.layer_1.weight)
        nn.init.zeros_(self.layer_1.bias)
        nn.init.xavier_uniform_(self.layer_2.weight)
        nn.init.zeros_(self.layer_2.bias)
        nn.init.xavier_uniform_(self.layer_out.weight)
        nn.init.zeros_(self.layer_out.bias)


    def forward(self, d):
        x = self.sigmoid(self.layer_1(d))
        # x = self.tanh(self.layer_2(x))
        x = self.softmax(self.layer_out(x))
        return x

model = SongModel(dataloader_train.dataset[0][0][0].size, dataloader_train.dataset[0][1][0].size).to(device)
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

In [27]:
def train(model, dataloader, optimizer,loss_fn):
    model.train()
    total_loss = 0
    iterations = 0
    acc = 0
    count = 0
    for sample in dataloader:
        model_input = sample[0][0]
        should = sample[1][0]
        predict = model(model_input)
        for sample_index in range(len(sample)):
            count += 1
            predict_e = predict[sample_index].data.numpy()
            should_e = should[sample_index].numpy()
            if np.where(predict_e == np.amax(predict_e))[0][0] == np.where(should_e == np.amax(should_e))[0][0]:
                acc += 1
        loss = loss_fn(predict, should.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss
        iterations += 1
        # if t % 10 == 0:
    print(f"\tTrain - \tLoss: {total_loss / len(dataloader):3.10}, Acc: {acc *100 / count:3.5}%", end="\n")

In [28]:
def test(model, dataloader, loss_fn):
    model.eval()
    total_loss = 0
    acc = 0
    count = 0
    genre_accuracy = {}
    for x, key in enumerate(lookup_dict):
        genre_accuracy[x] = 0
    
    for sample in dataloader:
        model_input = sample[0][0]
        should = sample[1][0]
        predict = model(model_input)
        for sample_index in range(len(sample)):
            #print(len(sample))
            count += 1
            predict_e = predict[sample_index].data.numpy()
            should_e = should[sample_index].numpy()
            if np.where(predict_e == np.amax(predict_e))[0][0] == np.where(should_e == np.amax(should_e))[0][0]:
                acc += 1
                genre_accuracy[np.where(should_e == np.amax(should_e))[0][0]] += 1
                
        loss = loss_fn(predict, should.float())
        total_loss += loss
        # if t % 10 == 0:
    #print(acc, count)
    print(f"\tEvaluation - \tLoss: {total_loss / len(dataloader):3.10}, Acc: {acc *100 / count:3.5}%", end="\n")
    #print(genre_accuracy)
    #for genre in lookup_dict:
        #print(f"\t {genre}:\t\t {round(genre_accuracy[lookup_dict[genre]] * 100 / genre_test_count_dict[np.where(should_e == np.amax(should_e))[0][0]],3)}%")

In [ ]:
epochs = 1000
for t in range(epochs):
    print("Epoch " + str(t) + ":")
    train(model, dataloader_train, optimizer, loss_fn)
    test(model, dataloader_test, loss_fn)
    torch.save(model.state_dict(), "model_test")

Epoch 0:
	Train - 	Loss: 1.85376358, Acc: 31.901%
	Evaluation - 	Loss: 1.850006938, Acc: 25.562%
Epoch 1:
	Train - 	Loss: 1.833557963, Acc: 33.732%
	Evaluation - 	Loss: 1.839122415, Acc: 26.124%
Epoch 2:
	Train - 	Loss: 1.824562788, Acc: 35.07%
	Evaluation - 	Loss: 1.833694339, Acc: 27.809%
Epoch 3:
	Train - 	Loss: 1.819427013, Acc: 35.423%
	Evaluation - 	Loss: 1.830513239, Acc: 28.371%
Epoch 4:
	Train - 	Loss: 1.816101551, Acc: 35.352%
	Evaluation - 	Loss: 1.828421831, Acc: 28.652%
Epoch 5:
	Train - 	Loss: 1.813737512, Acc: 35.845%
	Evaluation - 	Loss: 1.826940417, Acc: 29.213%
Epoch 6:
	Train - 	Loss: 1.811952233, Acc: 35.986%
	Evaluation - 	Loss: 1.825835466, Acc: 29.213%
Epoch 7:
	Train - 	Loss: 1.81054616, Acc: 35.704%
	Evaluation - 	Loss: 1.824979663, Acc: 29.213%
Epoch 8:
	Train - 	Loss: 1.809398055, Acc: 35.563%
	Evaluation - 	Loss: 1.824297309, Acc: 29.494%
Epoch 9:
	Train - 	Loss: 1.808443666, Acc: 35.493%
	Evaluation - 	Loss: 1.823740125, Acc: 29.494%
Epoch 10:
	Train - 	Los

In [ ]:
torch.save(model.state_dict(), "model_xxp")